In [141]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from keras import backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Dense, Activation

In [142]:
X = np.random.random((1000, 2))

In [143]:
X.shape

(1000, 2)

In [144]:
M = np.dot(X, [[2, 3, 1], 
               [4, 2, 1]]) + 1

In [145]:
M.shape

(1000, 3)

In [146]:
Y = np.dot(M, [[1], 
               [5], 
               [2]]) + 1

In [147]:
Y.shape

(1000, 1)

## base model

In [151]:
base_model = Sequential([
    Dense(3, input_shape=(2,))
])

In [152]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 3)                 9         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [153]:
base_model.compile(optimizer='sgd', loss='mse')
base_model.fit(X, M, epochs=10, verbose=0)

In [154]:
base_model.evaluate(X, M)

  32/1000 [..............................] - ETA: 1s

0.29504718255996704

freeze base_model weights

In [155]:
for layer in base_model.layers:
    layer.trainable = False

In [156]:
for layer in base_model.layers:
    print(layer.get_weights())

[array([[ 1.56013715,  0.63616467,  0.77701896],
       [ 2.27635026,  1.4363749 , -0.0208273 ]], dtype=float32), array([ 1.98713481,  2.37257767,  1.54214978], dtype=float32)]


In [157]:
pred_mid = base_model.predict(X)

In [158]:
base_model.layers[0].get_weights()

[array([[ 1.56013715,  0.63616467,  0.77701896],
        [ 2.27635026,  1.4363749 , -0.0208273 ]], dtype=float32),
 array([ 1.98713481,  2.37257767,  1.54214978], dtype=float32)]

In [159]:
pred_mid

array([[ 3.96763945,  3.54356599,  1.70285439],
       [ 3.16645336,  3.08085251,  1.61286569],
       [ 3.72132444,  3.26384521,  1.98750925],
       ..., 
       [ 3.86433411,  3.22607112,  2.27704644],
       [ 4.96236897,  4.03955603,  1.9929378 ],
       [ 3.95662689,  3.35061455,  2.12555265]], dtype=float32)

## base_model output as dense layer input

In [9]:
lm = Sequential([
    Dense(1, input_shape=(3,))
])
lm.compile(optimizer='sgd', loss='mse')

In [161]:
lm.fit(pred_mid, Y, epochs=10, verbose=1)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 25.4206     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 10.8088    
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 9.8470     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 8.9737      
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 8.4388     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 7.7182     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 7.2865     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 6.5749     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 6.1530     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 5.7681     


In [162]:
for layer in lm.layers:
    print(layer.get_weights())

[array([[ 6.51727343],
       [-0.74591929],
       [ 2.51487637]], dtype=float32), array([-0.66763282], dtype=float32)]


In [164]:
pred1 = lm.predict(pred_mid)

In [165]:
lm.evaluate(pred_mid, Y)

  32/1000 [..............................] - ETA: 0s

5.5476358985900882

## put in one model 

In [167]:
x = base_model.output
predictions = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='sgd', loss='mse')

In [168]:
model.fit(X, Y, epochs=10, verbose=1)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 35.8771     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 10.6723     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 9.9493      
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 9.0235     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 8.6559      
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 7.9104     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 7.2474     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 6.9621     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 6.4370     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 6.0261     


In [169]:
pred2 = model.predict(X)

In [170]:
model.evaluate(X, Y)

  32/1000 [..............................] - ETA: 1s

6.0149471664428713

In [171]:
for layer in model.layers:
    print(layer.get_weights())

[]
[array([[ 1.56013715,  0.63616467,  0.77701896],
       [ 2.27635026,  1.4363749 , -0.0208273 ]], dtype=float32), array([ 1.98713481,  2.37257767,  1.54214978], dtype=float32)]
[array([[ 5.9703145 ],
       [ 0.24839672],
       [ 1.92286444]], dtype=float32), array([-1.63435197], dtype=float32)]


In [172]:
base_model.layers[0].get_weights()

[array([[ 1.56013715,  0.63616467,  0.77701896],
        [ 2.27635026,  1.4363749 , -0.0208273 ]], dtype=float32),
 array([ 1.98713481,  2.37257767,  1.54214978], dtype=float32)]

In [173]:
np.all(model.layers[1].get_weights()[0] == base_model.layers[0].get_weights()[0])

True

In [174]:
np.all(model.layers[1].get_weights()[1] == base_model.layers[0].get_weights()[1])

True

In [175]:
model.layers[2].get_weights()

[array([[ 5.9703145 ],
        [ 0.24839672],
        [ 1.92286444]], dtype=float32), array([-1.63435197], dtype=float32)]

In [176]:
lm.layers[0].get_weights()

[array([[ 6.51727343],
        [-0.74591929],
        [ 2.51487637]], dtype=float32), array([-0.66763282], dtype=float32)]

### output of model at each layer

In [177]:
# https://stackoverflow.com/questions/41711190/keras-how-to-get-the-output-of-each-layer
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([inp]+ [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

# Testing
layer_outs = [func([X, 1.]) for func in functors]
print(layer_outs)

[[array([[ 0.22599092,  0.71514815],
       [ 0.10300355,  0.44747901],
       [ 0.58287656,  0.36234418],
       ..., 
       [ 0.95043391,  0.17325628],
       [ 0.60408664,  0.89299786],
       [ 0.76005018,  0.34428331]], dtype=float32)], [array([[ 3.96763945,  3.54356599,  1.70285439],
       [ 3.16645336,  3.08085251,  1.61286569],
       [ 3.72132444,  3.26384521,  1.98750925],
       ..., 
       [ 3.86433411,  3.22607112,  2.27704644],
       [ 4.96236897,  4.03955603,  1.9929378 ],
       [ 3.95662689,  3.35061455,  2.12555265]], dtype=float32)], [array([[ 26.20827293],
       [ 21.13696671],
       [ 25.21556473],
       [ 27.62999153],
       [ 24.60510826],
       [ 30.30875587],
       [ 16.98918152],
       [ 27.31812096],
       [ 19.22922516],
       [ 26.90186882],
       [ 27.55417442],
       [ 20.30547523],
       [ 24.77808189],
       [ 18.78953171],
       [ 28.2447052 ],
       [ 19.73711586],
       [ 17.46998405],
       [ 23.34695435],
       [ 29.33984184],

In [178]:
np.allclose(X, layer_outs[0])

True

In [179]:
np.all(pred_mid==layer_outs[1])

True

In [180]:
np.all(pred2==layer_outs[2])

True